<div style="font-size:18pt; padding-top:20px; text-align:center">СЕМИНАР. <b>Деревья решений и их ансамбли</b></div><hr>
<div style="text-align:right;">Папулин С.Ю. <span style="font-style: italic;font-weight: bold;">(papulin.study@yandex.ru)</span></div>

<a name="0"></a>
<div><span style="font-size:14pt; font-weight:bold">Содержание</span>
    <ol>
        <li><a href="#1">Дерево решений</a>
        <li><a href="#2">Ансамбль деревьев</a>
        <li><a href="#3">Источники</a>
        </li>
    </ol>
</div>

<p><b>Подключение библиотек</b></p>

In [ ]:
import time
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from scipy import stats

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
from sklearn.datasets import make_classification
from matplotlib import cm
from matplotlib.colors import ListedColormap

In [ ]:
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree

from sklearn.ensemble import (
    BaggingClassifier,
    BaggingRegressor,
    RandomForestClassifier,
    RandomForestRegressor,
    ExtraTreesClassifier,
    ExtraTreesRegressor
)

In [ ]:
import sys
sys.path.insert(0, "../lib/")
from plot_utils import CPlot, RPlot

<a name="1"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:16pt; font-weight:bold">1. Дерево решений</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

### Построение дерева решений для задачи регрессии вручную

In [ ]:
# Исходные данные
D = np.array([[2,1,1],
              [2,3,2],
              [5,3,2],
              [5,6,4],
              [6,5,4],
              [7,5,5],
              [8,7,6]])

X = D[:,:2]
y = D[:,2]

# График исходных данных
plt.figure(figsize=[6,6])
plt.scatter(X[:,0], X[:,1])
plt.title("Initial Data")
plt.xlabel("$X_1$")
plt.ylabel("$X_2$")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
def split(x, y, threshold):
    return y[x <= threshold], y[x > threshold]


def rss(left, right):
    return ((left - left.mean())**2).sum() + ((right - right.mean())**2).sum()


def branch_mse(y):
    return ((y - y.mean())**2).mean()


def print_info(split_point, rss):
    print("Split Point = {}\nRSS = {}".format(split_point, rss))

    
def calculate_splits(x, y):
    min_split = None
    min_rss = float("inf")
    x_set = np.sort(np.unique(x))
    for i in range(1, len(x_set)):
        split_point = x_set[i-1:i+1].mean()
        split_rss = rss(*split(x, y, split_point))
        print_info(split_point, split_rss)
        if min_rss > split_rss:
            min_rss = split_rss
            min_split = split_point
    return min_split, min_rss

Найти точку разделения с минимальной `RSS` по координатам `X1` и `X2`

In [ ]:
print("X1:\n")
min_split_point, min_rss = calculate_splits(X[:,0], y)
print("\nMin Split:\n\tSplit Point = {}\n\tRSS = {}\n\tMSE = {}".format(min_split_point, min_rss, branch_mse(y)))

In [ ]:
print("X2:\n")
min_split_point, min_rss = calculate_splits(X[:,1], y)
print("\nMin Split:\n\tSplit Point = {}\n\tRSS = {}\n\tMSE = {}".format(min_split_point, min_rss, branch_mse(y)))

Разделить исходные данные по `X2`

In [ ]:
X2_split_point = 4.0

# Индексы элементов
left_index = X[:,1] <= X2_split_point
left_index

In [ ]:
X2_split_point = 4.0

# Индексы элементов
left_index = X[:,1] <= X2_split_point
right_index = X[:,1] > X2_split_point

# Элементы левой части
X_left = X[left_index]
y_left = y[left_index]

# Элементы правой части
X_right = X[right_index]
y_right = y[right_index]

len(y_left), len(y_right)

В левой части только три элемента. Начальное условие не соблюдается, т.к. минимальное количество элементов в конечном узле должно быть не менее двух. Поэтому левую часть далее не делим.

In [ ]:
y_left_pred = y_left.mean()
y_left_pred

Найти точку разделения с минимальной `RSS` по координатам `X1` и `X2` для правой части

In [ ]:
print("X1:\n")
min_split_point, min_rss = calculate_splits(X_right[:,0], y_right)
print("\nMin Split:\n\tSplit Point = {}\n\tRSS = {}\n\tMSE = {}"
      .format(min_split_point, min_rss, branch_mse(y_right)))

In [ ]:
print("X2:\n")
min_split_point, min_rss = calculate_splits(X_right[:,1], y_right)
print("\nMin Split:\n\tSplit Point = {}\n\tRSS = {}\n\tMSE = {}"
      .format(min_split_point, min_rss, branch_mse(y_right)))

Разделить правую часть данные по `X1`

In [ ]:
X1_split_point = 6.5

# Индексы элементов
right_left_index = X_right[:,0] <= X1_split_point
right_right_index = X_right[:,0] > X1_split_point

# Элементы левой части
X_right_left = X_right[right_left_index]
y_right_left = y_right[right_left_index]

# Элементы правой части
X_right_right = X_right[right_right_index]
y_right_right = y_right[right_right_index]

len(y_right_left), len(y_right_right)

In [ ]:
y_right_left_pred = y_right_left.mean()
y_right_left_pred

In [ ]:
y_right_right_pred = y_right_right.mean()
y_right_right_pred

### Sklearn

In [ ]:
# Исходные данные
D = np.array([[2,1,1],
              [2,3,2],
              [5,3,2],
              [5,6,4],
              [6,5,4],
              [7,5,5],
              [8,7,6]])

X = D[:,:2]
y = D[:,2]

# График исходных данных
plt.figure(figsize=[6,6])
plt.scatter(X[:,0], X[:,1])
plt.title("Initial Data")
plt.xlabel("$X_1$")
plt.ylabel("$X_2$")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Обучение
tick = time.time()
dtr_model = DecisionTreeRegressor(criterion="mse", random_state=0, min_samples_leaf=2)
dtr_model.fit(X, y)
print("Time =", time.time()-tick)

# Отображение дерева решений
plt.figure(1, figsize=[14, 4])
plot_tree(dtr_model, filled=True, feature_names=["X1", "X2"])
plt.show()

### Регрессия

In [ ]:
# Исходные данные
n = 100
x = stats.uniform.rvs(size=n, loc=0, scale=5, random_state=0)
y = stats.norm.rvs(size=n, loc=0, scale=0.2, random_state=0) + np.sin(x)

RPlot.show_init_data_plot(x,y)

# Формирование обучающего и тестового подмножеств
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)
X_train = x_train[:, np.newaxis]
X_test = x_test[:, np.newaxis]

# Обучение
tick = time.time()
dtr_model = DecisionTreeRegressor(criterion="mse", max_depth=2, random_state=0)
dtr_model.fit(X_train, y_train)
print("Time =", time.time()-tick)

# Проверка на тестовом подмножестве
score_dtr_model = dtr_model.score(X_test, y_test)
print("R^2 =", score_dtr_model)

# Графики
RPlot.show_train_test_plots(dtr_model, X_train, y_train, X_test, y_test)

Структура дерева

In [ ]:
dtr_model.tree_

In [ ]:
plt.figure(1, figsize=[14, 4])
plot_tree(dtr_model, filled=True, feature_names=["X"])
plt.show()

### Классификация

In [ ]:
CLR_MAP = ListedColormap(["blue", "red", "green"])

In [ ]:
# Исходные данные
n = 500
X, y = make_classification(n_samples=n, n_features=2, n_redundant=0, 
                           n_informative=2, n_clusters_per_class=1, n_classes=3, class_sep=1,
                           random_state=1234)
CPlot.show_init_data_plot(X, y, cmap=CLR_MAP)

# Формирование обучающего и тестового подмножеств
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Обучение
tick = time.time()
dt_model = DecisionTreeClassifier(criterion="entropy", max_depth=2, random_state=0)
dt_model.fit(X_train, y_train)
print("Time = ", time.time()-tick)

# Проверка на тестовом подмножестве
score_dt_model = dt_model.score(X_test, y_test)
print("Test Score = ", score_dt_model)
print("Classification Report:")
print(classification_report(y_test, dt_model.predict(X_test), target_names=["Class 0", "Class 1", "Class 2"]))

# Графики
CPlot.show_train_test_plots(dt_model, X_train, y_train, X_test, y_test, 
                            title="Single Decision Tree", cmap=CLR_MAP)

Структура дерева

In [ ]:
dt_model.tree_

In [ ]:
plt.figure(1, figsize=[14, 8])
plot_tree(dt_model, filled=True, feature_names=["X1", "X2"], class_names=["0", "1", "2"])
plt.show()

<a name="2"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:16pt; font-weight:bold">2. Ансамбль деревьев</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

### Регрессия

In [ ]:
# Исходные данные
n = 100
x = stats.uniform.rvs(size=n, loc=0, scale=5, random_state=0)
y = stats.norm.rvs(size=n, loc=0, scale=0.2, random_state=0) + np.sin(x)

RPlot.show_init_data_plot(x, y)

In [ ]:
# Формирование обучающего и тестового подмножеств
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)
X_train = x_train[:, np.newaxis]
X_test = x_test[:, np.newaxis]

#### Дерево решений

In [ ]:
# Обучение
tick = time.time()
dtr_model = DecisionTreeRegressor(criterion="mse", max_depth=2, random_state=0)
dtr_model.fit(X_train, y_train)
print("Time =", time.time()-tick)

# Проверка на тестовом подмножестве
score_dtr_model = dtr_model.score(X_test, y_test)
print("R^2 =", score_dtr_model)
# TODO: MSE

#### Бэггинг

[BaggingRegressor](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingRegressor.html)

In [ ]:
# Обучение
tick = time.time()
dtr_model_inner = DecisionTreeRegressor(criterion="mse", max_depth=10, random_state=0)
baggingr_model = BaggingRegressor(base_estimator=dtr_model_inner, n_estimators=10, max_samples=1.0, 
                                  max_features=1.0, bootstrap=True, bootstrap_features=False, 
                                  oob_score=False, random_state=0)
baggingr_model.fit(X_train, y_train)
print("Time =", time.time()-tick)

# Проверка на тестовом подмножестве
score_baggingr_model = baggingr_model.score(X_test, y_test)
print("R^2 =", score_baggingr_model)

#### Случайный лес 

[RandomForestRegressor](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html)

In [ ]:
# Обучение
tick = time.time()
rfr_model = RandomForestRegressor(n_estimators=10, max_depth=10, criterion="mse", bootstrap=True, 
                                  max_features="sqrt", oob_score=False, random_state=0)
rfr_model.fit(X_train, y_train)
print("Time =", time.time()-tick)

# Проверка на тестовом подмножестве
score_rfr_model = rfr_model.score(X_test, y_test)
print("R^2 =", score_rfr_model)

#### Extra Trees

[ExtraTreesRegressor](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesRegressor.html)

In [ ]:
# Обучение
tick = time.time()
etr_model = ExtraTreesRegressor(n_estimators=10, max_depth=10, criterion="mse", bootstrap=True, 
                                max_features="sqrt", oob_score=False, random_state=0)
etr_model.fit(X_train, y_train)
print("Time =", time.time()-tick)

# Проверка на тестовом подмножестве
score_etr_model = etr_model.score(X_test, y_test)
print("R^2 =", score_etr_model)

#### Графики

In [ ]:
RPlot.show_train_test_plots(dtr_model, X_train, y_train, X_test, y_test, title="Single Decision Tree")
RPlot.show_train_test_plots(baggingr_model, X_train, y_train, X_test, y_test, title="Bagging")
RPlot.show_train_test_plots(rfr_model, X_train, y_train, X_test, y_test, title="Random Forest")
RPlot.show_train_test_plots(etr_model, X_train, y_train, X_test, y_test, title="Extra Trees")

### Классификация

In [ ]:
# Исходные данные
n = 500
X, y = make_classification(n_samples=n, n_features=2, n_redundant=0, 
                           n_informative=2, n_clusters_per_class=1, n_classes=3, class_sep=1,
                           random_state=1234)
CPlot.show_init_data_plot(X, y, cmap=CLR_MAP)

In [ ]:
# Формирование обучающего и тестового подмножеств
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

#### Дерево решений

In [ ]:
# Обучение
tick = time.time()
dt_model = DecisionTreeClassifier(criterion="entropy", random_state=0)
dt_model.fit(X_train, y_train)
print("Time =", time.time()-tick)

# Проверка на тестовом подмножестве
score_dt_model = dt_model.score(X_test, y_test)
print("Test Score = ", score_dt_model)

#### Бэггинг

[BaggingClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html)

In [ ]:
# Обучение
tick = time.time()
dt_model_inner = DecisionTreeClassifier(criterion="entropy", random_state=0)

bagging_model = BaggingClassifier(base_estimator=dt_model_inner, n_estimators=10, 
                                  max_samples=1.0, max_features=1.0, bootstrap=True, 
                                  bootstrap_features=False, oob_score=False, random_state=0)
bagging_model.fit(X_train, y_train)
print("Time =", time.time()-tick)

# Проверка на тестовом подмножестве
score_bagging_model = bagging_model.score(X_test, y_test)
print("Test Score = ", score_bagging_model)

#### Случайный лес 

[RandomForestClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

In [ ]:
# Обучение
tick = time.time()
rf_model = RandomForestClassifier(n_estimators=10, criterion="entropy", bootstrap=True, 
                                  oob_score=False, random_state=0)
rf_model.fit(X_train, y_train)
print("Time =", time.time()-tick)

# Проверка на тестовом подмножестве
score_rf_model = rf_model.score(X_test, y_test)
print("Test Score = ", score_rf_model)

#### Extra Trees

[ExtraTreesClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html)

In [ ]:
# Обучение
tick = time.time()
et_model = ExtraTreesClassifier(n_estimators=10, criterion="entropy", bootstrap=True, 
                                max_features="sqrt", oob_score=False, random_state=0)
et_model.fit(X_train, y_train)
print("Time =", time.time()-tick)

# Проверка на тестовом подмножестве
score_et_model = et_model.score(X_test, y_test)
print("Test Score = ", score_et_model)

#### Графики

In [ ]:
CPlot.show_train_test_plots(dt_model, X_train, y_train, X_test, y_test, title="Single Decision Tree", cmap=CLR_MAP)
CPlot.show_train_test_plots(bagging_model, X_train, y_train, X_test, y_test, title="Bagging", cmap=CLR_MAP)
CPlot.show_train_test_plots(rf_model, X_train, y_train, X_test, y_test, title="Random Forest", cmap=CLR_MAP)
CPlot.show_train_test_plots(et_model, X_train, y_train, X_test, y_test, title="Extra Trees", cmap=CLR_MAP)

<a name="3"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">3. Источники</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<a href="http://scikit-learn.org/stable/modules/tree.html">Decision Trees</a><br>
<a href="http://scikit-learn.org/stable/modules/ensemble.html">Ensemble methods</a><br>
<a href="http://nbviewer.jupyter.org/github/arundhaj/datamining_class/blob/master/decision_tree.ipynb">DecisionTree Classification</a>